In [ ]:
import tensorflow as tf
import cv2
import numpy as np
from tensorflow import keras
from pathlib import Path
from natsort import natsorted

In [ ]:
target_class={0:"Normal",1:"Tuberculosis"}

In [ ]:
Normal_dir=Path("TB_Chest_Radiography_Database/Normal/")
TB_dir=Path("TB_Chest_Radiography_Database/Tuberculosis/")

In [ ]:
Normal_images:list=natsorted(list(map(str, list(Normal_dir.glob("*.png")))))
TB_images:list=natsorted(list(map(str, list(TB_dir.glob("*.png")))))

In [ ]:
Normal_labels:list=[0]*len(Normal_images)
TB_labels:list=[1]*len(TB_images)

In [ ]:
images=np.array(Normal_images+TB_images)
labels=np.array(Normal_labels+TB_labels)
images.shape, labels.shape

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(images,labels,test_size=0.2,random_state=42)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
image_1=x_train[0]
image_1=cv2.imread(image_1)
lab=cv2.cvtColor(image_1,cv2.COLOR_BGR2LAB)
clahe=cv2.createCLAHE(2,(16,16))
lab[:, :, 0]=clahe.apply(lab[:, :, 0])
image_1_equalized=cv2.cvtColor(lab,cv2.COLOR_LAB2BGR)
image_1_equalized=cv2.cvtColor(image_1_equalized,cv2.COLOR_BGR2RGB)
image_1=cv2.cvtColor(image_1,cv2.COLOR_BGR2RGB)
_, ax = plt.subplots(1, 2, figsize=(7, 7))
ax[0].imshow(image_1)
ax[0].set_title("Before CLAHE")
ax[0].axis("off")
ax[1].imshow(image_1_equalized)
ax[1].set_title("After CLAHE")
ax[1].axis("off")

In [ ]:
def image_preprocessing(path):
    img=cv2.imread(path)
    img=cv2.cvtColor(image_1,cv2.COLOR_BGR2LAB)
    clahe=cv2.createCLAHE(2,(16,16))
    img[:, :, 0]=clahe.apply(img[:, :, 0])
    img=cv2.cvtColor(img,cv2.COLOR_LAB2BGR)
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    img=cv2.resize(img,(128,128))
    img=img/255.0
    return img

In [ ]:
x_train=np.array(list(map(image_preprocessing,x_train)))
x_train.shape

In [ ]:
x_valid=np.array(list(map(image_preprocessing,x_valid)))
x_valid.shape

In [ ]:
train_dataset=tf.data.Dataset.from_tensor_slices((x_train,y_train)).batch(10).prefetch(tf.data.AUTOTUNE)
validation_dataset=tf.data.Dataset.from_tensor_slices((x_valid,y_valid)).batch(10).prefetch(tf.data.AUTOTUNE)

In [ ]:
_, ax = plt.subplots(1, 10, figsize=(14, 4))
for batch in train_dataset.take(10):
    images = batch[0]
    labels = batch[1]
    for i in range(10):
        img = (images[i]*255).numpy().astype("uint8")
        label=labels[i].numpy()
        ax[i].imshow(img)
        ax[i].set_title(target_class[label])
        ax[i].axis("off")

In [ ]:
_, ax = plt.subplots(1, 10, figsize=(14, 4))
for batch in validation_dataset.take(10):
    images = batch[0]
    labels = batch[1]
    for i in range(10):
        img = (images[i]*255).numpy().astype("uint8")
        label=labels[i].numpy()
        ax[i].imshow(img)
        ax[i].set_title(target_class[label])
        ax[i].axis("off")

In [ ]:
class Patches(keras.layers.Layer):
    def __init__(self, patch_size):
        super().__init__()
        self.patch_size = patch_size

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(4, 4))
image = x_valid[np.random.choice(range(x_valid.shape[0]))]
plt.imshow((image*255).astype("uint8"))
plt.axis("off")

image=tf.convert_to_tensor([image])

patches = Patches(16)(image)
print(f"Image size: {256} X {256}")
print(f"Patch size: {16} X {16}")
print(f"Patches per image: {patches.shape[1]}")
print(f"Elements per patch: {patches.shape[-1]}")

n = int(np.sqrt(patches.shape[1]))
plt.figure(figsize=(4, 4))
for i, patch in enumerate(patches[0]):
    ax = plt.subplot(n, n, i + 1)
    patch_img = tf.reshape(patch, (16, 16, 3))
    plt.imshow((patch_img*255).numpy().astype("uint8"))
    plt.axis("off")

In [ ]:
class PatchEncoder(keras.layers.Layer):
    def __init__(self, num_patches, projection_dim):
        super().__init__()
        self.num_patches = num_patches
        self.projection = keras.layers.Dense(units=projection_dim)
        self.position_embedding = keras.layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded

In [ ]:
def mlp(x, hidden_units):
    for units in hidden_units:
        x = keras.layers.Dense(units, activation=tf.nn.gelu)(x)
    return x

In [ ]:
def build_model():
    inputs=keras.layers.Input(shape=(128,128,3))
    patches=Patches(16)(inputs)
    encoded=PatchEncoder(64,768)(patches)
    for _ in range(12):
        x1=keras.layers.LayerNormalization()(encoded)
        attention=keras.layers.MultiHeadAttention(num_heads=12,key_dim=768)(x1,x1)
        x2=keras.layers.Add()([attention,encoded])
        x3=keras.layers.LayerNormalization()(x2)
        x3=mlp(x3,[768])
        encoded=keras.layers.Add()([x3,x2])
    rep=keras.layers.Flatten()(encoded)
    rep=mlp(rep,hidden_units=[3072])
    output=keras.layers.Dense(1,activation='sigmoid')(rep)

    model=keras.Model(inputs=inputs,outputs=output)
    opt = keras.optimizers.Adam()
    model.compile(optimizer=opt,
                  loss=keras.losses.BinaryCrossentropy(),
                  metrics=keras.metrics.Recall())
    return model

In [ ]:
model=build_model()
model.summary()

In [ ]:
model.fit(train_dataset,validation_data=validation_dataset,epochs=10)